# Authentication System Migration to Cosmos DB

This notebook migrates the authentication system from in-memory data storage to Cosmos DB using MongoDB API.

## Overview
1. Setup Cosmos DB connection using CosmosDBMongoCoreManager
2. Insert sample policyholder data into Cosmos DB
3. Test data retrieval functionality
4. Update the authentication function to use Cosmos DB
5. Test the updated authentication system

## Prerequisites
- Cosmos DB instance with MongoDB API enabled
- Proper connection string configured in environment variables
- CosmosDBMongoCoreManager available in the project

## Import Required Libraries

In [1]:
import asyncio
import datetime
import sys
import os
from typing import Dict, List, Literal, Optional, TypedDict

import logging
import os

# set the directory to the location of the script
try:
    os.chdir("../../../")
    target_directory = os.getenv(
        "TARGET_DIRECTORY", os.getcwd()
    )  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        print(f"Changed directory to: {os.getcwd()}")
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

from src.cosmosdb.manager import CosmosDBMongoCoreManager
from utils.ml_logging import get_logger
from pymongo.errors import NetworkTimeout, DuplicateKeyError

logger = get_logger("auth_cosmos_migration")

Changed directory to: /Users/jinle/Repos/_AIProjects/art-voice-agent-accelerator


## Setup Cosmos DB Connection

In [2]:
# Initialize Cosmos DB Manager
def get_cosmos_manager() -> CosmosDBMongoCoreManager:
    """Initialize and return a Cosmos DB manager for authentication data."""
    cosmos_manager = CosmosDBMongoCoreManager(
        database_name="voice_agent_db",
        collection_name="policyholders"
    )
    return cosmos_manager

# Initialize the manager
try:
    cosmos = get_cosmos_manager()
    print("Cosmos DB Manager initialized successfully")
    print(f"Database: {cosmos.database.name}")
    print(f"Collection: {cosmos.collection.name}")
    print(f"Connection to cluster: {cosmos.cluster_host}")
except Exception as e:
    print(f"Failed to initialize Cosmos DB Manager: {e}")
    cosmos = None

Cosmos DB Manager initialized successfully
Database: voice_agent_db
Collection: policyholders
Connection to cluster: cosmos-cluster-yvy2hvjq.global.mongocluster.cosmos.azure.com


## Create Sample Policyholder Data

In [3]:
# Define the sample policyholder data (corrected to match policy ownership)
sample_policyholders = [
    {
        "_id": "jane_smith",
        "full_name": "Jane Smith",
        "zip": "60601",
        "ssn4": "5678",
        "policy4": "0001",  # Last 4 of POL-A10001
        "claim4": "9876",
        "phone4": "1078",
        "policy_id": "POL-A10001",  # Jane's Auto Insurance
        "created_at": datetime.datetime.utcnow().isoformat() + "Z",
        "updated_at": datetime.datetime.utcnow().isoformat() + "Z"
    },
    {
        "_id": "alice_brown",
        "full_name": "Alice Brown",
        "zip": "60601",
        "ssn4": "1234",  # Voice agent expects 1234
        "policy4": "0002",  # Last 4 of POL-A20002
        "claim4": "3344",
        "phone4": "4555",
        "policy_id": "POL-A20002",  # Alice's Home Insurance
        "created_at": datetime.datetime.utcnow().isoformat() + "Z",
        "updated_at": datetime.datetime.utcnow().isoformat() + "Z"
    },
    {
        "_id": "carlos_rivera",
        "full_name": "Carlos Rivera",
        "zip": "60601",
        "ssn4": "7890",
        "policy4": "4455",
        "claim4": "1122",
        "phone4": "9200",
        "policy_id": "POL-C88230",
        "created_at": datetime.datetime.utcnow().isoformat() + "Z",
        "updated_at": datetime.datetime.utcnow().isoformat() + "Z"
    }
]

print("Sample policyholder data:")
print("Data correlation with policy ownership:")
for holder in sample_policyholders:
    print(f"  - {holder['full_name']} → Policy: {holder['policy_id']}")

print("\nAuthentication-Policy Correlation:")
print("   - Jane Smith → POL-A10001 (Auto Insurance)")  
print("   - Alice Brown → POL-A20002 (Home Insurance)")
print("   - Carlos Rivera → POL-C88230 (existing)")

Sample policyholder data:
Data correlation with policy ownership:
  - Jane Smith → Policy: POL-A10001
  - Alice Brown → Policy: POL-A20002
  - Carlos Rivera → Policy: POL-C88230

Authentication-Policy Correlation:
   - Jane Smith → POL-A10001 (Auto Insurance)
   - Alice Brown → POL-A20002 (Home Insurance)
   - Carlos Rivera → POL-C88230 (existing)


## Insert Sample Data into Cosmos DB

In [4]:
async def insert_policyholder_data():
    """Insert sample policyholder data into Cosmos DB."""
    if not cosmos:
        print("Cosmos DB not initialized")
        return False
    
    try:
        # Insert each policyholder document
        for holder in sample_policyholders:
            try:
                # Use upsert to handle duplicates gracefully
                result = await asyncio.to_thread(
                    cosmos.upsert_document,
                    document=holder,
                    query={"_id": holder["_id"]}
                )
                print(f"Upserted: {holder['full_name']}")
            except Exception as e:
                print(f"Failed to insert {holder['full_name']}: {e}")
        
        print("All policyholder data inserted successfully!")
        return True
        
    except Exception as e:
        print(f"Failed to insert data: {e}")
        return False

# Run the insertion
if cosmos:
    await insert_policyholder_data()
else:
    print("Cannot insert data - Cosmos DB not available")

Upserted: Jane Smith
Upserted: Alice Brown
Upserted: Carlos Rivera
All policyholder data inserted successfully!


## Test Data Retrieval

In [5]:
async def get_policyholder_by_name(full_name: str) -> Optional[Dict]:
    """Retrieve a policyholder by full name from Cosmos DB."""
    if not cosmos:
        print("Cosmos DB not initialized")
        return None
    
    try:
        # Query by full_name field
        query = {"full_name": full_name.strip().title()}
        
        # Use read_document for a single document retrieval
        result = await asyncio.to_thread(
            cosmos.read_document,
            query=query
        )
        
        return result  # read_document returns the document or None
            
    except NetworkTimeout as err:
        print(f"Network timeout when querying for {full_name}: {err}")
        return None
    except Exception as e:
        print(f"Error querying for {full_name}: {e}")
        return None

# Test data retrieval
async def test_data_retrieval():
    """Test retrieving policyholder data from Cosmos DB."""
    test_names = ["Alice Brown", "Amelia Johnson", "Carlos Rivera", "NonExistent Person"]
    
    for name in test_names:
        print(f"\nSearching for: {name}")
        result = await get_policyholder_by_name(name)
        
        if result:
            print(f"Found: {result['full_name']} (Policy: {result['policy_id']})")
            print(f"   ZIP: {result['zip']}, Policy4: {result['policy4']}")
        else:
            print(f"Not found: {name}")

# Run the test
if cosmos:
    await test_data_retrieval()
else:
    print("Cannot test retrieval - Cosmos DB not available")


Searching for: Alice Brown
Found: Alice Brown (Policy: POL-A20002)
   ZIP: 60601, Policy4: 0002

Searching for: Amelia Johnson


Not found: Amelia Johnson

Searching for: Carlos Rivera
Found: Carlos Rivera (Policy: POL-C88230)
   ZIP: 60601, Policy4: 4455

Searching for: NonExistent Person


Not found: NonExistent Person


## Update Authentication Function

In [6]:
# Type definitions (from the original auth.py)
class AuthenticateArgs(TypedDict):
    """Payload expected by authenticate_caller."""
    full_name: str
    zip_code: str
    last4_id: str
    intent: Literal["claims", "general"]
    claim_intent: Optional[Literal["new_claim", "existing_claim", "unknown"]]
    attempt: Optional[int]

class AuthenticateResult(TypedDict):
    """Return schema from authenticate_caller."""
    authenticated: bool
    message: str
    policy_id: Optional[str]
    caller_name: Optional[str]
    attempt: int
    intent: Optional[Literal["claims", "general"]]
    claim_intent: Optional[Literal["new_claim", "existing_claim", "unknown"]]

async def authenticate_caller_cosmos(
    args: AuthenticateArgs,
    cosmos_manager: CosmosDBMongoCoreManager
) -> AuthenticateResult:
    """
    Updated authenticate_caller function that uses Cosmos DB instead of in-memory data.
    
    Parameters
    ----------
    args : AuthenticateArgs
        Authentication arguments including name, zip, last4, etc.
    cosmos_manager : CosmosDBMongoCoreManager
        Cosmos DB manager instance for data retrieval
        
    Returns
    -------
    AuthenticateResult
        Authentication result with success/failure status
    """
    # Input type validation to prevent 400 errors
    if not isinstance(args, dict):
        logger.error("Invalid args type: %s. Expected dict.", type(args))
        return {
            "authenticated": False,
            "message": "Invalid request format. Please provide authentication details.",
            "policy_id": None,
            "caller_name": None,
            "attempt": 1,
            "intent": None,
            "claim_intent": None,
        }

    # Extract and validate inputs
    zip_code = args.get("zip_code", "").strip() if args.get("zip_code") else ""
    last4_id = args.get("last4_id", "").strip() if args.get("last4_id") else ""

    if not zip_code and not last4_id:
        msg = "zip_code or last4_id must be provided"
        logger.error("%s", msg)
        attempt = int(args.get("attempt", 1))
        return {
            "authenticated": False,
            "message": msg,
            "policy_id": None,
            "caller_name": None,
            "attempt": attempt,
            "intent": None,
            "claim_intent": None,
        }

    # Normalize inputs
    full_name = (
        args.get("full_name", "").strip().title() if args.get("full_name") else ""
    )
    last4 = last4_id
    attempt = int(args.get("attempt", 1))

    if not full_name:
        logger.error("full_name is required")
        return {
            "authenticated": False,
            "message": "Full name is required for authentication.",
            "policy_id": None,
            "caller_name": None,
            "attempt": attempt,
            "intent": None,
            "claim_intent": None,
        }

    intent = args.get("intent", "general")
    claim_intent = args.get("claim_intent")

    logger.info(
        "Attempt %d – Authenticating %s | ZIP=%s | last-4=%s | intent=%s | claim_intent=%s",
        attempt,
        full_name,
        zip_code or "<none>",
        last4 or "<none>",
        intent,
        claim_intent,
    )

    # Query Cosmos DB for the policyholder
    try:
        rec = await get_policyholder_by_name(full_name)
    except Exception as e:
        logger.error("Database error during authentication for %s: %s", full_name, e)
        return {
            "authenticated": False,
            "message": "Authentication service temporarily unavailable. Please try again.",
            "policy_id": None,
            "caller_name": None,
            "attempt": attempt,
            "intent": None,
            "claim_intent": None,
        }

    if not rec:
        logger.warning("Name not found: %s", full_name)
        return {
            "authenticated": False,
            "message": f"Name '{full_name}' not found.",
            "policy_id": None,
            "caller_name": None,
            "attempt": attempt,
            "intent": None,
            "claim_intent": None,
        }

    # Validate ZIP code and last-4 identifiers
    last4_fields: List[str] = ["ssn4", "policy4", "claim4", "phone4"]
    last4_match = bool(last4) and last4 in (rec[f] for f in last4_fields)
    zip_match = bool(zip_code) and rec["zip"] == zip_code

    if zip_match or last4_match:
        logger.info("Authentication succeeded for %s", full_name)
        return {
            "authenticated": True,
            "message": f"Authenticated {full_name}.",
            "policy_id": rec["policy_id"],
            "caller_name": full_name,
            "attempt": attempt,
            "intent": intent,
            "claim_intent": claim_intent,
        }

    # Authentication failed
    logger.warning("ZIP and last-4 both mismatched for %s", full_name)
    return {
        "authenticated": False,
        "message": "Authentication failed - ZIP and last-4 did not match.",
        "policy_id": None,
        "caller_name": None,
        "attempt": attempt,
        "intent": None,
        "claim_intent": None,
    }

print("Updated authentication function created!")

Updated authentication function created!


## Test Updated Authentication

In [8]:
async def test_authentication_scenarios():
    """Test the updated authentication function with various scenarios."""
    if not cosmos:
        print("Cannot test authentication - Cosmos DB not available")
        return
    
    test_cases = [
        # Successful authentication with ZIP
        {
            "name": "Success with ZIP",
            "args": {
                "full_name": "Alice Brown",
                "zip_code": "60601",
                "last4_id": "",
                "intent": "claims",
                "claim_intent": "new_claim",
                "attempt": 1
            },
            "expected": True
        },
        
        # Successful authentication with last-4 SSN
        {
            "name": "Success with SSN last-4",
            "args": {
                "full_name": "Amelia Johnson",
                "zip_code": "",
                "last4_id": "5566",
                "intent": "general",
                "claim_intent": None,
                "attempt": 1
            },
            "expected": True
        },
        
        # Failed authentication - wrong ZIP
        {
            "name": "Failed with wrong ZIP",
            "args": {
                "full_name": "Carlos Rivera",
                "zip_code": "12345",
                "last4_id": "",
                "intent": "claims",
                "claim_intent": "existing_claim",
                "attempt": 1
            },
            "expected": False
        },
        
        # Failed authentication - wrong last-4
        {
            "name": "Failed with wrong last-4",
            "args": {
                "full_name": "Alice Brown",
                "zip_code": "",
                "last4_id": "9999",
                "intent": "general",
                "claim_intent": None,
                "attempt": 2
            },
            "expected": False
        },
        
        # Failed authentication - user not found
        {
            "name": "User not found",
            "args": {
                "full_name": "Unknown Person",
                "zip_code": "60601",
                "last4_id": "",
                "intent": "claims",
                "claim_intent": "new_claim",
                "attempt": 1
            },
            "expected": False
        },
        
        # Error case - missing required fields
        {
            "name": "Missing verification data",
            "args": {
                "full_name": "Alice Brown",
                "zip_code": "",
                "last4_id": "",
                "intent": "general",
                "claim_intent": None,
                "attempt": 1
            },
            "expected": False
        }
    ]
    
    print("Testing authentication scenarios with Cosmos DB:")
    print("=" * 60)
    
    for test_case in test_cases:
        print(f"\nTest: {test_case['name']}")
        print(f"   Input: {test_case['args']['full_name']} | ZIP: {test_case['args']['zip_code'] or 'None'} | Last-4: {test_case['args']['last4_id'] or 'None'}")
        
        try:
            result = await authenticate_caller_cosmos(test_case["args"], cosmos)
            
            success = result["authenticated"]
            expected = test_case["expected"]
            
            if success == expected:
                status = "PASS"
            else:
                status = "FAIL"
            
            print(f"   Result: {status} | Authenticated: {success} | Message: {result['message']}")
            if success:
                print(f"           Policy ID: {result['policy_id']} | Intent: {result['intent']}")
            
        except Exception as e:
            print(f"   ERROR: {e}")
    
    print("\nAuthentication testing completed!")

# Run the tests
if cosmos:
    await test_authentication_scenarios()
else:
    print("Cannot run tests - Cosmos DB not available")

Testing authentication scenarios with Cosmos DB:

Test: Success with ZIP
   Input: Alice Brown | ZIP: 60601 | Last-4: None


[2025-10-20 21:14:12,264] INFO - auth_cosmos_migration: Attempt 1 – Authenticating Alice Brown | ZIP=60601 | last-4=<none> | intent=claims | claim_intent=new_claim
INFO:auth_cosmos_migration:Attempt 1 – Authenticating Alice Brown | ZIP=60601 | last-4=<none> | intent=claims | claim_intent=new_claim
[2025-10-20 21:14:12,309] INFO - auth_cosmos_migration: Authentication succeeded for Alice Brown
INFO:auth_cosmos_migration:Authentication succeeded for Alice Brown


   Result: PASS | Authenticated: True | Message: Authenticated Alice Brown.
           Policy ID: POL-A20002 | Intent: claims

Test: Success with SSN last-4
   Input: Amelia Johnson | ZIP: None | Last-4: 5566


[2025-10-20 21:14:12,313] INFO - auth_cosmos_migration: Attempt 1 – Authenticating Amelia Johnson | ZIP=<none> | last-4=5566 | intent=general | claim_intent=None
INFO:auth_cosmos_migration:Attempt 1 – Authenticating Amelia Johnson | ZIP=<none> | last-4=5566 | intent=general | claim_intent=None
[2025-10-20 21:14:12,355] WARNING - auth_cosmos_migration: Name not found: Amelia Johnson


   Result: FAIL | Authenticated: False | Message: Name 'Amelia Johnson' not found.

Test: Failed with wrong ZIP
   Input: Carlos Rivera | ZIP: 12345 | Last-4: None


...

   Result: PASS | Authenticated: False | Message: Authentication failed - ZIP and last-4 did not match.

Test: User not found
   Input: Unknown Person | ZIP: 60601 | Last-4: None


[2025-10-20 21:14:12,455] INFO - auth_cosmos_migration: Attempt 1 – Authenticating Unknown Person | ZIP=60601 | last-4=<none> | intent=claims | claim_intent=new_claim
INFO:auth_cosmos_migration:Attempt 1 – Authenticating Unknown Person | ZIP=60601 | last-4=<none> | intent=claims | claim_intent=new_claim
[2025-10-20 21:14:12,501] WARNING - auth_cosmos_migration: Name not found: Unknown Person


   Result: PASS | Authenticated: False | Message: Name 'Unknown Person' not found.

Test: Missing verification data
   Input: Alice Brown | ZIP: None | Last-4: None


[2025-10-20 21:14:12,504] ERROR - auth_cosmos_migration: zip_code or last4_id must be provided
ERROR:auth_cosmos_migration:zip_code or last4_id must be provided


   Result: PASS | Authenticated: False | Message: zip_code or last4_id must be provided

Authentication testing completed!


## Production-Ready Compound Query Approach

In [9]:
# Additional test data with duplicate names
duplicate_name_policyholders = [
    {
        "_id": "alice_brown_chicago",
        "full_name": "Alice Brown",
        "zip": "60622",  # Different ZIP (Chicago)
        "ssn4": "5678",
        "policy4": "0002",
        "claim4": "4321",
        "phone4": "2468",
        "policy_id": "POL-A20002",
        "created_at": datetime.datetime.utcnow().isoformat() + "Z",
        "updated_at": datetime.datetime.utcnow().isoformat() + "Z"
    },
    {
        "_id": "alice_brown_milwaukee",
        "full_name": "Alice Brown",
        "zip": "53201",  # Different ZIP (Milwaukee)
        "ssn4": "9999",
        "policy4": "0003",
        "claim4": "2222",
        "phone4": "3333",
        "policy_id": "POL-A30003",
        "created_at": datetime.datetime.utcnow().isoformat() + "Z",
        "updated_at": datetime.datetime.utcnow().isoformat() + "Z"
    }
]

print("Duplicate name test data:")
print("Each Alice Brown maps to her correct policy:")
for holder in duplicate_name_policyholders:
    print(f"  - {holder['full_name']} (ZIP: {holder['zip']}) → Policy: {holder['policy_id']}")

print("\nComplete Alice Brown Mapping:")
print("   - Alice Brown (60601) → Original (gets corrected below)")
print("   - Alice Brown (60622) → POL-A20002 (Home Insurance)")  
print("   - Alice Brown (53201) → POL-A30003 (Life Insurance)")

# Insert the duplicate name data
async def insert_duplicate_name_data():
    """Insert duplicate name data to test production scenario."""
    if not cosmos:
        print("Cosmos DB not initialized")
        return False
    
    try:
        for holder in duplicate_name_policyholders:
            try:
                result = await asyncio.to_thread(
                    cosmos.upsert_document,
                    document=holder,
                    query={"_id": holder["_id"]}
                )
                print(f"Upserted: {holder['full_name']} (ZIP: {holder['zip']}) → {holder['policy_id']}")
            except Exception as e:
                print(f"Failed to insert {holder['full_name']}: {e}")
        
        print("Duplicate name test data inserted successfully!")
        return True
        
    except Exception as e:
        print(f"Failed to insert duplicate data: {e}")
        return False

# Insert the data
if cosmos:
    await insert_duplicate_name_data()
else:
    print("Cannot insert data - Cosmos DB not available")

Duplicate name test data:
Each Alice Brown maps to her correct policy:
  - Alice Brown (ZIP: 60622) → Policy: POL-A20002
  - Alice Brown (ZIP: 53201) → Policy: POL-A30003

Complete Alice Brown Mapping:
   - Alice Brown (60601) → Original (gets corrected below)
   - Alice Brown (60622) → POL-A20002 (Home Insurance)
   - Alice Brown (53201) → POL-A30003 (Life Insurance)
Upserted: Alice Brown (ZIP: 60622) → POL-A20002
Upserted: Alice Brown (ZIP: 53201) → POL-A30003
Duplicate name test data inserted successfully!


In [10]:
# Production-ready compound query function
async def _get_policyholder_by_credentials_production(cosmos_manager, full_name: str, **kwargs) -> Optional[dict]:
    """
    Production-ready implementation that handles duplicate names.
    Uses compound queries to find unique matches.
    
    Args:
        cosmos_manager: The Cosmos DB manager instance
        full_name: The caller's full name
        **kwargs: Additional credentials (zip, ssn4, policy4, claim4, phone4)
    
    Returns:
        dict: Policyholder data if unique match found, None otherwise
    """
    if not cosmos_manager:
        raise ValueError("Cosmos DB manager not available")
    
    # First, check how many people have this name
    name_query = {"full_name": full_name}
    
    try:
        all_matches = await asyncio.to_thread(
            cosmos_manager.find_documents,
            query=name_query
        )
        
        print(f"Found {len(all_matches)} people named '{full_name}'")
        
        if len(all_matches) == 0:
            print(f"No policyholder found with name: {full_name}")
            return None
        
        elif len(all_matches) == 1:
            print(f"Unique name match found for: {full_name}")
            return all_matches[0]
        
        else:
            # Multiple people with same name - need additional credentials
            print(f"Multiple people named '{full_name}' - using compound query...")
            
            # Build compound query conditions
            compound_conditions = []
            
            # ZIP code verification
            if "zip" in kwargs and kwargs["zip"]:
                compound_conditions.append({"zip": kwargs["zip"]})
            
            # Last-4 matching (could be SSN, policy, claim, or phone)
            if "last4" in kwargs and kwargs["last4"]:
                last4_conditions = [
                    {"ssn4": kwargs["last4"]},
                    {"policy4": kwargs["last4"]},
                    {"claim4": kwargs["last4"]},
                    {"phone4": kwargs["last4"]}
                ]
                compound_conditions.append({"$or": last4_conditions})
            
            if not compound_conditions:
                print("Multiple matches found but no additional credentials provided")
                return None
            
            # Create final compound query
            compound_query = {
                "$and": [
                    {"full_name": full_name},
                    *compound_conditions
                ]
            }
            
            print(f"Compound query: {compound_query}")
            
            # Execute compound query
            compound_matches = await asyncio.to_thread(
                cosmos_manager.find_documents,
                query=compound_query
            )
            
            if len(compound_matches) == 1:
                print(f"Unique compound match found for: {full_name}")
                return compound_matches[0]
            elif len(compound_matches) == 0:
                print(f"No matches found with additional credentials")
                return None
            else:
                print(f"Still multiple matches ({len(compound_matches)}) even with additional credentials")
                return None
                
    except Exception as e:
        print(f"Database error during compound query: {e}")
        return None

print("Production compound query function defined!")
print("This function handles:")
print("   - Unique name matches (single result)")
print("   - Multiple name matches (compound queries)")
print("   - ZIP code verification")
print("   - Last-4 matching across multiple fields")

Production compound query function defined!
This function handles:
   - Unique name matches (single result)
   - Multiple name matches (compound queries)
   - ZIP code verification
   - Last-4 matching across multiple fields


In [11]:
# Test scenarios for production compound queries
test_scenarios = [
    {
        "name": "Scenario 1: Unique Name Match",
        "description": "Testing with a name that has only one match",
        "test_data": {
            "full_name": "Jane Smith",
        },
        "expected": "Should find unique match"
    },
    {
        "name": "Scenario 2: Duplicate Names - ZIP Resolution",
        "description": "Multiple Alice Browns, resolved by ZIP code",
        "test_data": {
            "full_name": "Alice Brown",
            "zip": "60622"
        },
        "expected": "Should find Chicago Alice Brown"
    },
    {
        "name": "Scenario 3: Duplicate Names - Last-4 SSN Resolution",
        "description": "Multiple Alice Browns, resolved by SSN last-4",
        "test_data": {
            "full_name": "Alice Brown",
            "last4": "9999"
        },
        "expected": "Should find Milwaukee Alice Brown"
    },
    {
        "name": "Scenario 4: Duplicate Names - Last-4 Policy Resolution",
        "description": "Multiple Alice Browns, resolved by Policy last-4",
        "test_data": {
            "full_name": "Alice Brown",
            "last4": "8765"
        },
        "expected": "Should find Chicago Alice Brown via policy"
    },
    {
        "name": "Scenario 5: No Match Found",
        "description": "Testing with non-existent name",
        "test_data": {
            "full_name": "Non Existent Person"
        },
        "expected": "Should return None"
    },
    {
        "name": "Scenario 6: Ambiguous Credentials",
        "description": "Multiple matches with insufficient credentials",
        "test_data": {
            "full_name": "Alice Brown"
        },
        "expected": "Should return None (multiple matches, no disambiguation)"
    }
]

print("Production Test Scenarios Defined:")
for i, scenario in enumerate(test_scenarios, 1):
    print(f"   {i}. {scenario['name']}")
    print(f"      {scenario['description']}")
    print(f"      {scenario['expected']}")
    print()

Production Test Scenarios Defined:
   1. Scenario 1: Unique Name Match
      Testing with a name that has only one match
      Should find unique match

   2. Scenario 2: Duplicate Names - ZIP Resolution
      Multiple Alice Browns, resolved by ZIP code
      Should find Chicago Alice Brown

   3. Scenario 3: Duplicate Names - Last-4 SSN Resolution
      Multiple Alice Browns, resolved by SSN last-4
      Should find Milwaukee Alice Brown

   4. Scenario 4: Duplicate Names - Last-4 Policy Resolution
      Multiple Alice Browns, resolved by Policy last-4
      Should find Chicago Alice Brown via policy

   5. Scenario 5: No Match Found
      Testing with non-existent name
      Should return None

   6. Scenario 6: Ambiguous Credentials
      Multiple matches with insufficient credentials
      Should return None (multiple matches, no disambiguation)



In [12]:
# Execute all production test scenarios
async def run_production_tests():
    """Run comprehensive production scenario tests."""
    if not cosmos:
        print("Cosmos DB not available for testing")
        return
    
    print("Running Production Compound Query Tests")
    print("=" * 60)
    
    results = []
    
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"\nTest {i}: {scenario['name']}")
        print(f"{scenario['description']}")
        print(f"Test Data: {scenario['test_data']}")
        print(f"Expected: {scenario['expected']}")
        print("-" * 40)
        
        try:
            # Execute the test
            result = await _get_policyholder_by_credentials_production(
                cosmos, 
                **scenario['test_data']
            )
            
            # Analyze result
            if result:
                print(f"Result: Found policyholder")
                print(f"   Name: {result.get('full_name')}")
                print(f"   ZIP: {result.get('zip')}")
                print(f"   Policy: {result.get('policy_id')}")
                success = True
            else:
                print(f"Result: No policyholder found")
                success = True  # This might be expected for some tests
            
            results.append({
                "scenario": scenario['name'],
                "success": success,
                "result": result
            })
            
        except Exception as e:
            print(f"Test failed with error: {e}")
            results.append({
                "scenario": scenario['name'],
                "success": False,
                "error": str(e)
            })
    
    # Summary
    print("\n" + "=" * 60)
    print("PRODUCTION TEST SUMMARY")
    print("=" * 60)
    
    successful_tests = sum(1 for r in results if r['success'])
    total_tests = len(results)
    
    print(f"Successful Tests: {successful_tests}/{total_tests}")
    
    for result in results:
        status = "PASS" if result['success'] else "FAIL"
        print(f"{status} {result['scenario']}")
        if 'error' in result:
            print(f"    Error: {result['error']}")
    
    print(f"\nProduction testing complete!")
    return results

# Run the production tests
if cosmos:
    production_results = await run_production_tests()
else:
    print("Cannot run production tests - Cosmos DB not available")

Running Production Compound Query Tests

Test 1: Scenario 1: Unique Name Match
Testing with a name that has only one match
Test Data: {'full_name': 'Jane Smith'}
Expected: Should find unique match
----------------------------------------
Database error during compound query: 'CosmosDBMongoCoreManager' object has no attribute 'find_documents'
Result: No policyholder found

Test 2: Scenario 2: Duplicate Names - ZIP Resolution
Multiple Alice Browns, resolved by ZIP code
Test Data: {'full_name': 'Alice Brown', 'zip': '60622'}
Expected: Should find Chicago Alice Brown
----------------------------------------
Database error during compound query: 'CosmosDBMongoCoreManager' object has no attribute 'find_documents'
Result: No policyholder found

Test 3: Scenario 3: Duplicate Names - Last-4 SSN Resolution
Multiple Alice Browns, resolved by SSN last-4
Test Data: {'full_name': 'Alice Brown', 'last4': '9999'}
Expected: Should find Milwaukee Alice Brown
----------------------------------------
Data